# Develop a LangGraph Agent

- 任意の LangGraph エージェントを AgentEngine でラップする
- https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/develop/langgraph#langgraph


In [13]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [14]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'/home/mori/.gcloud/agent-engine-test-461308-a2228f8932d9.json'

## Vertex AI の初期化


In [4]:
import vertexai

vertexai.init(
    project="agent-engine-test-461308",
    location="us-central1",  # Agent Engine にデプロイするときに使用する Bucket
    staging_bucket="gs://agent-engine-test-250529",
)

## LangGraph エージェントを構成する

- シンプルな build 済みの create_react_agent を利用
- langgraph_builder で囲む
- https://langchain-ai.github.io/langgraph/agents/agents/


In [11]:
from langgraph.prebuilt import create_react_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


def langgraph_builder(model, *, tools, **kwargs):
    # LanggraphAgentで渡されるmodelやtoolsを利用してReActエージェントを作成
    return create_react_agent(
        model=model,
        tools=tools,
    )

## ローカルでエージェントを起動する


In [7]:
from vertexai import agent_engines

# Geminiを使う場合
agent = agent_engines.LanggraphAgent(
    model="gemini-2.0-flash-lite-001",  # usでしか使えない
    tools=[get_weather],
    runnable_builder=langgraph_builder,
)

In [ ]:
# OpenAIを使う場合
# from langchain_openai import ChatOpenAI


# def model_builder(*, model_name, model_kwargs=None, **kwargs):
#     return ChatOpenAI(model=model_name, **model_kwargs)


# agent = agent_engines.LanggraphAgent(
#     model="gpt-4.1-mini",
#     model_builder=model_builder,
#     tools=[get_weather],
#     runnable_builder=langgraph_builder,
#     model_kwargs={
#         "api_key": os.environ["OPENAI_API_KEY"],
#         "temperature": 0,
#     },
# )

In [21]:
# シンプルなクエリ
response = agent.query(
    input={"messages": [("user", "やあ")]},
)
response

{'messages': [{'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'kwargs': {'content': 'やあ',
    'type': 'human',
    'id': 'd790424c-49d3-4afe-ad8a-c5104823eac0'}},
  {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'kwargs': {'content': 'こんにちは！何かお手伝いできることはありますか？\n',
    'response_metadata': {'is_blocked': False,
     'safety_ratings': [],
     'usage_metadata': {'prompt_token_count': 16,
      'candidates_token_count': 11,
      'total_token_count': 27,
      'prompt_tokens_details': [{'modality': 1, 'token_count': 16}],
      'candidates_tokens_details': [{'modality': 1, 'token_count': 11}],
      'thoughts_token_count': 0,
      'cached_content_token_count': 0,
      'cache_tokens_details': []},
     'finish_reason': 'STOP',
     'avg_logprobs': -0.03513369506055659,
     'model_name': 'gemini-2.0-flash-lite-001'},
    'type': 'ai',
    'id': 'run--134fdc22-9276-482d-85e9-5688da74e15c

In [22]:
# ツール利用
response = agent.query(
    input={"messages": [("user", "今日の東京の天気は？")]},
)
response

{'messages': [{'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'kwargs': {'content': '今日の東京の天気は？',
    'type': 'human',
    'id': '45657b9a-dcea-4302-b432-b1bbe3761343'}},
  {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'kwargs': {'content': '',
    'additional_kwargs': {'function_call': {'name': 'get_weather',
      'arguments': '{"city": "\\u6771\\u4eac"}'}},
    'response_metadata': {'is_blocked': False,
     'safety_ratings': [],
     'usage_metadata': {'prompt_token_count': 20,
      'candidates_token_count': 5,
      'total_token_count': 25,
      'prompt_tokens_details': [{'modality': 1, 'token_count': 20}],
      'candidates_tokens_details': [{'modality': 1, 'token_count': 5}],
      'thoughts_token_count': 0,
      'cached_content_token_count': 0,
      'cache_tokens_details': []},
     'finish_reason': 'STOP',
     'avg_logprobs': -0.00016216778894886374,
     'model_name':

In [23]:
# イベントのストリーム表示
for chunk in agent.stream_query(
    input={"messages": [("user", "今日の東京の天気は？")]},
):
    print(chunk)
    print("=" * 100)

{'agent': {'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_weather', 'arguments': '{"city": "\\u6771\\u4eac"}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 20, 'candidates_token_count': 5, 'total_token_count': 25, 'prompt_tokens_details': [{'modality': 1, 'token_count': 20}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.00017849390860646963, 'model_name': 'gemini-2.0-flash-lite-001'}, 'type': 'ai', 'id': 'run--65532000-a1b5-467b-961e-e7b6f0ff1396-0', 'tool_calls': [{'name': 'get_weather', 'args': {'city': '東京'}, 'id': '79b30c20-e217-43ec-831b-9364cfd58a28', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 20, 'output_tokens': 5, 'to

In [24]:
# トークンのストリーム表示
for chunk in agent.stream_query(
    input={"messages": [("user", "今日の東京の天気は？")]},
    stream_mode=["updates", "messages"],
):
    print(chunk)
    print("=" * 100)

['messages', [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_weather', 'arguments': '{"city": "\\u6771\\u4eac"}'}}, 'response_metadata': {'safety_ratings': [], 'usage_metadata': {}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite-001'}, 'type': 'AIMessageChunk', 'id': 'run--6d5b1d37-370f-4d5c-84f4-e96f200742d4', 'tool_calls': [{'name': 'get_weather', 'args': {'city': '東京'}, 'id': 'e6a9cc89-50c4-476e-9882-aa16c5bae300', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 20, 'output_tokens': 5, 'total_tokens': 25}, 'tool_call_chunks': [{'name': 'get_weather', 'args': '{"city": "\\u6771\\u4eac"}', 'id': 'e6a9cc89-50c4-476e-9882-aa16c5bae300', 'index': None, 'type': 'tool_call_chunk'}], 'invalid_tool_calls': []}}, {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ['branch:to:agent'], 'langgraph_path': ['__pregel_pull', 'agen

## エージェントのデプロイ

- IAM ロール `ストレージ管理者` を追加する


In [15]:
from vertexai.preview.reasoning_engines import ReasoningEngine

remote_agent = ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle",
        "langgraph",
        "pydantic",
        "langchain-openai",
    ],
    display_name="Simple ReAct Agent",
    description="A simple ReAct agent with LangGraph",
    extra_packages=[],
)

Using bucket agent-engine-test-250529
Writing to gs://agent-engine-test-250529/reasoning_engine/reasoning_engine.pkl
Writing to gs://agent-engine-test-250529/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://agent-engine-test-250529/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/714913803218/locations/us-central1/reasoningEngines/8888038582569140224/operations/3442924227756294144
ReasoningEngine created. Resource name: projects/714913803218/locations/us-central1/reasoningEngines/8888038582569140224
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/714913803218/locations/us-central1/reasoningEngines/8888038582569140224')


# デプロイしたエージェントを確認する


In [16]:
from vertexai import agent_engines

for resource in agent_engines.list():
    print(resource)

resource name: projects/714913803218/locations/us-central1/reasoningEngines/8888038582569140224


# デプロイしたエージェントを使用する


In [17]:
remote_agent = agent_engines.get(
    "projects/714913803218/locations/us-central1/reasoningEngines/8888038582569140224"
)

In [18]:
# シンプルなクエリ
response = remote_agent.query(
    input={"messages": [("user", "やあ")]},
)
response

{'messages': [{'kwargs': {'content': 'やあ',
    'type': 'human',
    'id': '1ad1ceeb-d383-4482-8e09-596699e25cb5'},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'lc': 1.0},
  {'kwargs': {'content': 'こんにちは！何かお手伝いできることはありますか？\n',
    'tool_calls': [],
    'type': 'ai',
    'id': 'run--1af04432-1c92-470b-8dfe-394d458fec5f-0',
    'response_metadata': {'safety_ratings': [],
     'finish_reason': 'STOP',
     'avg_logprobs': -0.03513369506055659,
     'usage_metadata': {'prompt_tokens_details': [{'modality': 1.0,
        'token_count': 16.0}],
      'total_token_count': 27.0,
      'cache_tokens_details': [],
      'candidates_tokens_details': [{'modality': 1.0, 'token_count': 11.0}],
      'cached_content_token_count': 0.0,
      'prompt_token_count': 16.0,
      'candidates_token_count': 11.0,
      'thoughts_token_count': 0.0},
     'is_blocked': False,
     'model_name': 'gemini-2.0-flash-lite-001'},
    'usage_metadata': {'output_tokens': 11

In [19]:
# ツール利用
response = remote_agent.query(
    input={"messages": [("user", "今日の東京の天気は？")]},
)
response

{'messages': [{'kwargs': {'content': '今日の東京の天気は？',
    'type': 'human',
    'id': 'ad60eb97-f9f0-4df0-adef-2ead175e6d1a'},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'lc': 1.0},
  {'kwargs': {'content': '',
    'tool_calls': [{'type': 'tool_call',
      'args': {'city': '東京'},
      'id': '664eed58-846f-4fec-8c31-dc15408336e7',
      'name': 'get_weather'}],
    'type': 'ai',
    'id': 'run--87038fc5-aa06-4d07-baf1-fe0d6bf3cf25-0',
    'response_metadata': {'safety_ratings': [],
     'finish_reason': 'STOP',
     'avg_logprobs': -0.00016216778894886374,
     'usage_metadata': {'prompt_tokens_details': [{'modality': 1.0,
        'token_count': 20.0}],
      'total_token_count': 25.0,
      'cache_tokens_details': [],
      'candidates_tokens_details': [{'modality': 1.0, 'token_count': 5.0}],
      'cached_content_token_count': 0.0,
      'prompt_token_count': 20.0,
      'candidates_token_count': 5.0,
      'thoughts_token_count': 0.0},
  

In [21]:
# トークンのストリーム表示
for chunk in remote_agent.stream_query(
    input={"messages": [("user", "今日の東京の天気は？")]},
    stream_mode=["updates", "messages"],
):
    print(chunk)
    print("=" * 100)

['messages', [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_weather', 'arguments': '{"city": "\\u6771\\u4eac"}'}}, 'response_metadata': {'safety_ratings': [], 'usage_metadata': {}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite-001'}, 'type': 'AIMessageChunk', 'id': 'run--7db3ec2e-2389-47f8-a607-34b0b5dbc108', 'tool_calls': [{'name': 'get_weather', 'args': {'city': '東京'}, 'id': 'fcb9d4df-3768-4a2d-b3fb-5da44a79e8e9', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 20, 'output_tokens': 5, 'total_tokens': 25}, 'tool_call_chunks': [{'name': 'get_weather', 'args': '{"city": "\\u6771\\u4eac"}', 'id': 'fcb9d4df-3768-4a2d-b3fb-5da44a79e8e9', 'index': None, 'type': 'tool_call_chunk'}], 'invalid_tool_calls': []}}, {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ['branch:to:agent'], 'langgraph_path': ['__pregel_pull', 'agen